In [1]:
import lerobot
from lerobot.common.datasets import lerobot_dataset

In [2]:
from pathlib import Path
from pprint import pprint

# import imageio
import torch
# import torch dataset
from torch.utils.data import DataLoader, Dataset

import lerobot
from lerobot.common.datasets.lerobot_dataset import LeRobotDataset

# print("List of available datasets:")
# pprint(lerobot.available_datasets)

# Let's take one for this example
repo_id = "lerobot/pusht"

# You can easily load a dataset from a Hugging Face repository
dataset = LeRobotDataset(repo_id)

# LeRobotDataset is actually a thin wrapper around an underlying Hugging Face dataset
# (see https://huggingface.co/docs/datasets/index for more information).
print(dataset)
print(dataset.hf_dataset)

# And provides additional utilities for robotics and compatibility with Pytorch
print(f"\naverage number of frames per episode: {dataset.num_samples / dataset.num_episodes:.3f}")
print(f"frames per second used during data collection: {dataset.fps=}")
print(f"keys to access images from cameras: {dataset.camera_keys=}\n")

Fetching 212 files:   0%|          | 0/212 [00:00<?, ?it/s]

LeRobotDataset(
  Repository ID: 'lerobot/pusht',
  Split: 'train',
  Number of Samples: 25650,
  Number of Episodes: 206,
  Type: video (.mp4),
  Recorded Frames per Second: 10,
  Camera Keys: ['observation.image'],
  Video Frame Keys: ['observation.image'],
  Transformations: None,
  Codebase Version: v1.6,
)
Dataset({
    features: ['observation.image', 'observation.state', 'action', 'episode_index', 'frame_index', 'timestamp', 'next.reward', 'next.done', 'next.success', 'index'],
    num_rows: 25650
})

average number of frames per episode: 124.515
frames per second used during data collection: dataset.fps=10
keys to access images from cameras: dataset.camera_keys=['observation.image']



In [3]:
# Access frame indexes associated to first episode
episode_index = 0
from_idx = dataset.episode_data_index["from"][episode_index].item()
to_idx = dataset.episode_data_index["to"][episode_index].item()

# LeRobot datasets actually subclass PyTorch datasets so you can do everything you know and love from working
# with the latter, like iterating through the dataset. Here we grab all the image frames.
frames = [dataset[idx]["observation.image"] for idx in range(from_idx, to_idx)]
states = [dataset[idx]["observation.state"] for idx in range(from_idx, to_idx)]
actions = [dataset[idx]["action"] for idx in range(from_idx, to_idx)]
timestamps = [dataset[idx]["timestamp"] for idx in range(from_idx, to_idx)]

# Video frames are now float32 in range [0,1] channel first (c,h,w) to follow pytorch convention. To visualize
# them, we convert to uint8 in range [0,255]
# frames = [(frame * 255).type(torch.uint8) for frame in frames]
# # and to channel last (h,w,c).
# frames = [frame.permute((1, 2, 0)).numpy() for frame in frames]

# # Finally, we save the frames to a mp4 video for visualization.
# Path("outputs/examples/1_load_lerobot_dataset").mkdir(parents=True, exist_ok=True)
# imageio.mimsave("outputs/examples/1_load_lerobot_dataset/episode_0.mp4", frames, fps=dataset.fps)

# # For many machine learning applications we need to load the history of past observations or trajectories of
# # future actions. Our datasets can load previous and future frames for each key/modality, using timestamps
# # differences with the current loaded frame. For instance:
# delta_timestamps = {
#     # loads 4 images: 1 second before current frame, 500 ms before, 200 ms before, and current frame
#     "observation.image": [-1, -0.5, -0.20, 0],
#     # loads 8 state vectors: 1.5 seconds before, 1 second before, ... 20 ms, 10 ms, and current frame
#     "observation.state": [-1.5, -1, -0.5, -0.20, -0.10, -0.02, -0.01, 0],
#     # loads 64 action vectors: current frame, 1 frame in the future, 2 frames, ... 63 frames in the future
#     "action": [t / dataset.fps for t in range(64)],
# }
# dataset = LeRobotDataset(repo_id, delta_timestamps=delta_timestamps)
# print(f"\n{dataset[0]['observation.image'].shape=}")  # (4,c,h,w)
# print(f"{dataset[0]['observation.state'].shape=}")  # (8,c)
# print(f"{dataset[0]['action'].shape=}\n")  # (64,c)

# # Finally, our datasets are fully compatible with PyTorch dataloaders and samplers because they are just
# # PyTorch datasets.
# dataloader = torch.utils.data.DataLoader(
#     dataset,
#     num_workers=0,
#     batch_size=32,
#     shuffle=True,
# )
# for batch in dataloader:
#     print(f"{batch['observation.image'].shape=}")  # (32,4,c,h,w)
#     print(f"{batch['observation.state'].shape=}")  # (32,8,c)
#     print(f"{batch['action'].shape=}")  # (32,64,c)
#     break

MemoryError: [Errno 12] Cannot allocate memory

In [5]:
for frame, state, action, ts in zip(frames, states, actions, timestamps):
    print(frame.shape)
    print(state.shape)
    print(action.shape)
    print(ts)
    break

torch.Size([3, 96, 96])
torch.Size([2])
torch.Size([2])
tensor(0.)


In [6]:
for i in range(15):
    print(states[i])

tensor([222.,  97.])
tensor([225.2524,  89.3125])
tensor([227.5923,  84.5344])
tensor([228.4202,  84.2799])
tensor([229.0422,  84.9571])
tensor([232.1624,  86.3440])
tensor([238.8461,  89.3548])
tensor([248.0901,  94.0601])
tensor([258.1464,  99.5915])
tensor([268.2669, 105.9949])
tensor([278.6407, 114.0329])
tensor([288.4110, 123.1640])
tensor([295.9132, 130.9943])
tensor([302.2147, 138.0031])
tensor([308.5594, 144.0331])


In [3]:
fps = 10
obs_steps = 2
horizon = 16

delta_timestamps = {
    "observation.image": [i / fps for i in range(1 - obs_steps, 1)],
    "observation.state": [i / fps for i in range(1 - obs_steps, 1)],
    "action": [i / fps for i in range(1 - obs_steps, 1 - obs_steps + horizon)],
  }

delta_timestamps

{'observation.image': [-0.1, 0.0],
 'observation.state': [-0.1, 0.0],
 'action': [-0.1,
  0.0,
  0.1,
  0.2,
  0.3,
  0.4,
  0.5,
  0.6,
  0.7,
  0.8,
  0.9,
  1.0,
  1.1,
  1.2,
  1.3,
  1.4]}

In [6]:
dataset = LeRobotDataset(repo_id, delta_timestamps=delta_timestamps)
# print(f"\n{dataset[0]['observation.image'].shape=}")  # (4,c,h,w)
# print(f"{dataset[0]['observation.state'].shape=}")  # (8,c)
# print(f"{dataset[0]['action'].shape=}\n")  # (64,c)

Fetching 212 files:   0%|          | 0/212 [00:00<?, ?it/s]

In [9]:
episode_index = 0
from_idx = dataset.episode_data_index["from"][episode_index].item()
to_idx = dataset.episode_data_index["to"][episode_index + 2].item()
first_rollout = [dataset[idx] for idx in range(from_idx, to_idx)]

MemoryError: [Errno 12] Cannot allocate memory

In [8]:
print(to_idx)
# print(first_rollout[0].keys())

420


In [10]:
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]
    
sub_dataset = CustomDataset(first_rollout)

# class FilteredDataset(Dataset):
#     def __init__(self, dataset, condition):
#         self.dataset = dataset
#         self.condition = condition
#         self.indices = [i for i, item in enumerate(dataset) if condition(item)]
    
#     def __len__(self):
#         return len(self.indices)
    
#     def __getitem__(self, idx):
#         original_idx = self.indices[idx]
#         return self.dataset[original_idx]

# # Define the filtering condition
# condition = lambda x: x["episode_index"] == 0

NameError: name 'first_rollout' is not defined

In [25]:
# Apply the filter
# first_rollout = FilteredDataset(dataset, condition)

In [11]:
episode_index = 0
from_idx = sub_dataset.episode_data_index["from"][episode_index].item()

NameError: name 'sub_dataset' is not defined

In [26]:
i = 2
max_horizon= 20
action_horizon = 16
obs_horizon = 2
for j in range(i + 1, max_horizon):
    if j > i + action_horizon - obs_horizon + 1:
        print(j)

18
19


In [34]:
print(len(first_rollout))
print(first_rollout[0].keys())

279
dict_keys(['observation.image', 'observation.state', 'action', 'episode_index', 'frame_index', 'timestamp', 'next.reward', 'next.done', 'next.success', 'index', 'observation.image_is_pad', 'observation.state_is_pad', 'action_is_pad'])


In [44]:
class EnrichedRobotDataset(torch.utils.data.Dataset):
    """Enriched dataset that includes subsequent goals of the original expert dataset."""

    def __init__(self, original_dataset, goal_horizon, lerobot_dataset):
        """Initialize the dataset.

        Inputs:
            original_dataset: the original dataset to enrich (obtained from LeRobot for example)
            goal_horizon: the number of steps to look ahead for the goal (must be at least action_horizon - obs_horizon + 1)
        """
        self.original_dataset = original_dataset
        self.enriched_data = []
        self.goal_horizon = goal_horizon
        self.lerobot_dataset = lerobot_dataset
        self._enrich_dataset()

    def _enrich_dataset(self):
        n = len(self.original_dataset)
        for i in range(n):
            item = self.original_dataset[i]
            agent_pos = item["observation.state"] # (obs_horizon, 2)
            action = item["action"] # (action_horizon, 2)
            image = item["observation.image"] # (obs_horizon, C, H, W)

            episode_index = item["episode_index"]
            current_episode_end = self.lerobot_dataset.episode_data_index["to"][episode_index].item()
            action_horizon = action.shape[0]
            obs_horizon = agent_pos.shape[0]
            if self.goal_horizon is None:
                self.goal_horizon = current_episode_end
            assert self.goal_horizon > action_horizon - obs_horizon + 1, (
                f"goal_horizon ({self.goal_horizon}) must be greater than "
                f"action_horizon ({action_horizon}) - obs_horizon ({obs_horizon}) + 1"
            )
            # iteration at most until the end of the current episode
            max_horizon = min(i + self.goal_horizon, current_episode_end)
            for j in range(i + 1, max_horizon):
                # add future goals at least action_horizon - obs_horizon + 1 steps ahead and beyond
                if j > i + action_horizon - obs_horizon + 1:
                # print(f"Enriching dataset: {i}/{n} - {j}/{n}")
                    future_goal_pos = self.original_dataset[j]["observation.state"]
                    future_goal_image = self.original_dataset[j]["observation.image"]
                    self.enriched_data.append(
                        {
                            "agent_pos": agent_pos,
                            "action": action,
                            "image": image,
                            "reached_goal_agent_pos": future_goal_pos,
                            "reached_goal_image": future_goal_image,
                        }
                    )

    def __len__(self):
        """Return the number of samples in the dataset."""
        return len(self.enriched_data)

    def __getitem__(self, idx):
        """Get a sample from the dataset."""
        return self.enriched_data[idx]

In [46]:
test_dataset = EnrichedRobotDataset(first_rollout, goal_horizon=30, lerobot_dataset=dataset)

for i in range(len(test_dataset)):
    print(test_dataset[i]["agent_pos"].shape)
    print(test_dataset[i]["action"].shape)
    print(test_dataset[i]["image"].shape)
    print(test_dataset[i]["reached_goal_agent_pos"].shape)
    print(test_dataset[i]["reached_goal_image"].shape)
    break

print(len(test_dataset))

torch.Size([2, 2])
torch.Size([16, 2])
torch.Size([2, 3, 96, 96])
torch.Size([2, 2])
torch.Size([2, 3, 96, 96])
3276


In [43]:
test_dataset[0]["reached_goal_agent_pos"] == sub_dataset[16]["observation.state"]

tensor([[True, True],
        [True, True]])

In [84]:
print(test_dataset[0]["reached_goal_agent_pos"])

tensor([[314.8816, 149.7178],
        [321.6607, 155.1989]])


In [86]:
print(test_dataset[16]["agent_pos"])

tensor([[228.4202,  84.2799],
        [229.0422,  84.9571]])


In [13]:
import torch

if torch.cuda.is_available():
    print(f"CUDA is available. Device count: {torch.cuda.device_count()}")
    print(f"CUDA Device Name: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available.")

CUDA is not available.
